<!-- NOTEBOOK_METADATA source: "Jupyter Notebook" title: "Example - Pydantic AI MCP Agent Evaluation" description: "This guide shows how to evaluate Pydantic AI MCP Agents with Langfuse using online and offline evaluation methods." category: "Evaluation" -->

# Langfuse × Pydantic AI – Agent Evals

## 1. Setup – install packages & add credentials

In [1]:
# If you are running this on colab/@home comment‑out what you already have
%pip install -q --upgrade "pydantic-ai[mcp]" langfuse openai nest_asyncio aiohttp

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os

# Get keys for your project from the project settings page: https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf..." 
os.environ["LANGFUSE_SECRET_KEY"] = "sk-..." 
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

# Your openai key
os.environ["OPENAI_API_KEY"] = "sk-proj-..."

## 2. Enable Langfuse Tracing

All integrations: https://langfuse.com/integrations

In [3]:
from langfuse import get_client
from pydantic_ai.agent import Agent

# Initialise Langfuse client and verify connectivity
langfuse = get_client()
assert langfuse.auth_check(), "Langfuse auth failed - check your keys ✋"

# Turn on OpenTelemetry instrumentation for *all* future Agent instances
Agent.instrument_all()
print("✅ Pydantic AI instrumentation enabled - traces will stream to Langfuse")

✅ Pydantic AI instrumentation enabled - traces will stream to Langfuse


## 3. Create an agent that can search the Langfuse docs

We use the Lagfuse Docs MCP Server to provide tools to the agent: https://langfuse.com/docs/docs-mcp

In [9]:
from pydantic_ai import Agent, RunContext
from pydantic_ai.mcp import MCPServerStreamableHTTP, CallToolFunc, ToolResult
from typing import Any

# Public MCP server that exposes Langfuse docs tools
LANGFUSE_MCP_URL = "https://langfuse.com/api/mcp"

async def run_agent(item, system_prompt="You are an expert on Langfuse. ", model="openai:o3-mini"):
    langfuse.update_current_trace(input=item.input)

    tool_call_history = []

    # Log all tool calls for trajectory analysis
    async def process_tool_call(
        ctx: RunContext[int],
        call_tool: CallToolFunc,
        tool_name: str,
        args: dict[str, Any],
    ) -> ToolResult:
        """A tool call processor that passes along the deps."""
        print(f"MCP Tool call: {tool_name} with args: {args}")
        tool_call_history.append({
            "tool_name": tool_name,
            "args": args
        })
        return await call_tool(tool_name, args)
    
    langfuse_docs_server = MCPServerStreamableHTTP(
        LANGFUSE_MCP_URL,
        process_tool_call=process_tool_call
    )

    agent = Agent(
        model=model,
        mcp_servers=[langfuse_docs_server],
        system_prompt=system_prompt
    )

    async with agent.run_mcp_servers():
        print("\n---")
        print("Q:", item.input)
        result = await agent.run(item.input["question"])
        print("A:", result.output)

        langfuse.update_current_trace(
            output=result.output,
            metadata={"tool_call_history": tool_call_history
        })

        return result.output, tool_call_history

## Evaluation

1. Create Test Cases
    - input
    - reference for reference-based evaluations
2. Set up evaluators
3. Run experiments

In [16]:
tests_cases = [
    {
        "input": {"question": "What is Langfuse?"},
        "expected_output": {
            "response_facts": [
                "Open Source LLM Engineering Platform",
                "Product modules: Tracing, Evaluation and Prompt Management"
            ],
            "trajectory": [
                "getLangfuseOverview"
            ],
        }
    },
    {
        "input": {
            "question": "How to trace a python application with Langfuse?"
        },
        "expected_output": {
            "response_facts": [
                "Python SDK, you can use the observe() decorator",
                "Lots of integrations, LangChain, LlamaIndex, Pydantic AI, and many more."
            ],
            "trajectory": [
                "getLangfuseOverview",
                "searchLangfuseDocs"
            ],
            "search_term": "Python Tracing"
        }
    },
    {
        "input": {"question": "How to connect to the Langfuse Docs MCP server?"},
        "expected_output": {
            "response_facts": [
                "Connect via the MCP server endpoint: https://langfuse.com/api/mcp",
                "Transport protocol: `streamableHttp`"
            ],
            "trajectory": ["getLangfuseOverview"]
        }
    },
    {
        "input": {
            "question": "How long are traces retained in langfuse?",
        },
        "expected_output": {
            "response_facts": [
                "By default, traces are retained indefinetly",
                "You can set custom data retention policy in the project settings"
            ],
            "trajectory": ["getLangfuseOverview", "searchLangfuseDocs"],
            "search_term": "Data retention"
        }
    }
]

Upload to Langfuse datasets

In [5]:
DATSET_NAME = "pydantic-ai-mcp-agent-evaluation"

In [ ]:
dataset = langfuse.create_dataset(
    name=DATSET_NAME
)
for case in tests_cases:
    langfuse.create_dataset_item(
        dataset_name=DATSET_NAME,
        input=case["input"],
        expected_output=case["expected_output"]
    )

### Set up Evaluations in Langfuse

#### Final response evaluation

```md
You are a teacher grading a student based on the factual correctness of his statements. In the following please find some example gradings that you did in the past.

### Examples

#### **Example 1:**
- **Response:** "The sun is shining brightly."
- **Facts to verify:** ["The sun is up.", "It is a beautiful day."]

Grading
- Reasoning: The response accurately includes both facts and aligns with the context of a beautiful day.
- Score: 1

#### **Example 2:**
- **Response:** "When I was in the kitchen, the dog was there"
- **Facts to verify:** ["The cat is on the table.", "The dog is in the kitchen."]

Grading
- Reasoning: The response includes that the dog is in the kitchen but does not mention that the cat is on the table.
- Score: 0

### New Student Response

- **Response**: {{response}}
- **Facts to verify:** {{facts_to_verify}}
```

#### Trajectory

```md
You are comparing two lists of strings. Please check whether the lists contain exactly the same items. The order does not matter.

## Examples

Input
Expected: ["searchWeb", "visitWebsite"]
Output: ["searchWeb"]

Grading
Reasoning: ["searchWeb", "visitWebsite"] are expected. In the output, "visitWebsite" is missing. Thus the two arrays are not the same.
Score: 0

Input
Expected: ["drawImage", "visitWebsite", "speak"]
Output: ["visitWebsite", "speak", "drawImage"]

Grading
Reasoning: The output matches the items from the expected output.
Score: 1

Input
Expected: ["getNews"]
Output: ["getNews", "watchTv"]

Grading
Reasoning: The output contains "watchTv" which was not expected.
Score: 0

## This excercise

Expected: {{expected}}
Output: {{output}}
```

#### Search quality

```md
You are a teacher grading a student based on whether he has looked for the right information in order to answe a question. In the following please find some example gradings that you did in the past.

The search by the student does not need to exactly match the response you expected; searches are often brief. The search term should correspond vaguely with the expected search term.

### Examples
#### **Example 1:**
- **Response:** How can I contact support?
- **Expected search topics**: Support

Grading
- Reasoning: The response accurately searches for support.
- Score: 1

#### **Example 2:**
- **Response:** Deployment
- **Expected search topics:** Tracing

Grading
- Reasoning: The response does not match the expected search topic of Tracing. Deployment questions are unrelated.
- Score: 0

#### **Example 3:**
- **Response:**
- **Expected search topics:**

Grading
- Reasoning: No search was done and no search term was expected.
- Score: 1

#### **Example 4:**
- **Response:** How to view sessions?
- **Expected search topics:**

Grading
- Reasoning: No search was expected, but search was used. This is not a problem.
- Score: 1

#### **Example 5:**
- **Response:**
- **Expected search topics:** How to run Langfuse locally?

Grading
- Reasoning: Even though we expected a search regarding running Langfuse locally, no search was made.
- Score: 0

### New Student Response

- **Response:** {{search}}
- **Expected search topics:** {{expected_search_topic}}
```

### Run Experiments

In [ ]:
system_prompts = {
    "simple": (
        "You are an expert on Langfuse. "
        "Answer user questions accurately and concisely using the available MCP tools. "
        "Cite sources when appropriate."
    ),
    "nudge_search_and_sources": (
        "You are an expert on Langfuse. "
        "Answer user questions accurately and concisely using the available MCP tools. "
        "Always cite sources when appropriate."
        "When you are unsure, always use getLangfuseOverview tool to do some research and then search the docs for more information. You can if needed use these tools multiple times."
    )
}

models = [
    "openai:gpt-5-nano",
    "openai:gpt-5-mini"
]

In [ ]:
# Get dataset from Langfuse
dataset = langfuse.get_dataset(DATSET_NAME)

# Run experiment directly on the dataset
result = dataset.run_experiment(
    name="Production Model Test"
    description="Monthly evaluation of our production model",
    task=run_agent # see above for the task definition
)

# Use format method to display results
print(result.format())

In [11]:
from functools import partial

# Get dataset from Langfuse
dataset = langfuse.get_dataset("pydantic-ai-mcp-agent-evaluation")


for prompt_name, prompt_content in list(system_prompts.items()):
    for test_model in models:

        task = partial(
            run_agent,
            system_prompt=prompt_content,
            model=test_model,
        )

        # Run experiment directly on the dataset
        result = dataset.run_experiment(
            name="Production Model Test: " + prompt_name + " " + test_model,
            description="Monthly evaluation of our production model",
            task=task # see above for the task definition
        )

        # Use format method to display results
        print(result.format())



---
Q: {'question': 'How long are traces retained in langfuse?'}

---
Q: {'question': 'How to connect to the Langfuse Docs MCP server?'}

---
Q: {'question': 'How to trace a python application with Langfuse?'}

---
Q: {'question': 'What is Langfuse?'}
MCP Tool call: searchLangfuseDocs with args: {'query': 'trace a Python application with Langfuse'}
MCP Tool call: searchLangfuseDocs with args: {'query': 'trace retention period in Langfuse'}
MCP Tool call: searchLangfuseDocs with args: {'query': 'What is Langfuse?'}
MCP Tool call: searchLangfuseDocs with args: {'query': 'Connecting to the Langfuse MCP server'}
A: In Langfuse, traces are retained for a configurable period specified at the project level, with a minimum retention period of 3 days. By default, data retention is set to indefinite storage unless explicitly configured otherwise. The retention policy applies to traces, observations, scores, and media assets and is managed through project settings or API for non-zero values. You

Item 2 failed: unhandled errors in a TaskGroup (1 sub-exception)
Item 3 failed: unhandled errors in a TaskGroup (1 sub-exception)


Individual Results: Hidden (2 items)\n💡 Set include_item_results=True to view them\n\n──────────────────────────────────────────────────\n🧪 Experiment: Production Model Test: simple openai:gpt-4.1-nano
📋 Run name: Production Model Test: simple openai:gpt-4.1-nano - 2025-11-05T10:44:50.974380Z - Monthly evaluation of our production model\n2 items\n🔗 Dataset Run:\n   https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/datasets/cmhlmusje003fad07bzz4bhim/runs/86101eb4-b41e-4666-9bc9-8045de24c241

---
Q: {'question': 'How to connect to the Langfuse Docs MCP server?'}

---
Q: {'question': 'What is Langfuse?'}

---
Q: {'question': 'How long are traces retained in langfuse?'}

---
Q: {'question': 'How to trace a python application with Langfuse?'}
MCP Tool call: searchLangfuseDocs with args: {'query': 'What is Langfuse? Langfuse overview'}
MCP Tool call: searchLangfuseDocs with args: {'query': 'trace a python application with Langfuse'}
MCP Tool call: searchLangfuseDocs with args: {'q

Item 1 failed: unhandled errors in a TaskGroup (1 sub-exception)
Item 3 failed: unhandled errors in a TaskGroup (1 sub-exception)


Individual Results: Hidden (2 items)\n💡 Set include_item_results=True to view them\n\n──────────────────────────────────────────────────\n🧪 Experiment: Production Model Test: simple openai:o4-mini
📋 Run name: Production Model Test: simple openai:o4-mini - 2025-11-05T10:49:52.742937Z - Monthly evaluation of our production model\n2 items\n🔗 Dataset Run:\n   https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/datasets/cmhlmusje003fad07bzz4bhim/runs/b97f1ae5-d7fa-4186-93b9-d340f6e78ba1

---
Q: {'question': 'How long are traces retained in langfuse?'}

---
Q: {'question': 'What is Langfuse?'}

---
Q: {'question': 'How to trace a python application with Langfuse?'}

---
Q: {'question': 'How to connect to the Langfuse Docs MCP server?'}
MCP Tool call: searchLangfuseDocs with args: {'query': 'trace retention period in langfuse'}
MCP Tool call: searchLangfuseDocs with args: {'query': 'What is Langfuse?'}
MCP Tool call: searchLangfuseDocs with args: {'query': 'trace a Python application

Item 0 failed: unhandled errors in a TaskGroup (1 sub-exception)
Item 2 failed: unhandled errors in a TaskGroup (1 sub-exception)
Item 3 failed: unhandled errors in a TaskGroup (1 sub-exception)


Individual Results: Hidden (1 items)\n💡 Set include_item_results=True to view them\n\n──────────────────────────────────────────────────\n🧪 Experiment: Production Model Test: nudge_search_and_sources openai:gpt-4.1-nano
📋 Run name: Production Model Test: nudge_search_and_sources openai:gpt-4.1-nano - 2025-11-05T10:55:03.366739Z - Monthly evaluation of our production model\n1 items\n🔗 Dataset Run:\n   https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/datasets/cmhlmusje003fad07bzz4bhim/runs/0d13d393-1ba4-4fa3-a109-78b28de59b52

---
Q: {'question': 'What is Langfuse?'}

---
Q: {'question': 'How long are traces retained in langfuse?'}

---
Q: {'question': 'How to connect to the Langfuse Docs MCP server?'}

---
Q: {'question': 'How to trace a python application with Langfuse?'}
MCP Tool call: getLangfuseOverview with args: {}
MCP Tool call: searchLangfuseDocs with args: {'query': 'traces retention length in langfuse traces retained'}
MCP Tool call: searchLangfuseDocs with args: {

Item 0 failed: unhandled errors in a TaskGroup (1 sub-exception)
Item 2 failed: unhandled errors in a TaskGroup (1 sub-exception)


Individual Results: Hidden (2 items)\n💡 Set include_item_results=True to view them\n\n──────────────────────────────────────────────────\n🧪 Experiment: Production Model Test: nudge_search_and_sources openai:o4-mini
📋 Run name: Production Model Test: nudge_search_and_sources openai:o4-mini - 2025-11-05T11:00:05.606758Z - Monthly evaluation of our production model\n2 items\n🔗 Dataset Run:\n   https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/datasets/cmhlmusje003fad07bzz4bhim/runs/235549d8-d116-4532-ae29-2f030cebc4cf
